# Multi-tenant Chat with Papers - Setup
## Get keys and urls

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_URL = os.getenv("OPENAI_URL")

print(f"Weaviate Key:{WEAVIATE_KEY}")
print(f"OpenAI API Key: {OPENAI_API_KEY}")
print(f"OpenAI URL: {OPENAI_URL}")

## Connect to Weaviate

In [ ]:
import weaviate
from weaviate.classes.init import Auth

# Connect to the local instance
client = weaviate.connect_to_local(
  host="127.0.0.1", # the address to the learner's instance
  port=8080,
  grpc_port=50051,
  auth_credentials=Auth.api_key(WEAVIATE_KEY),
  headers={
    "X-OpenAI-Api-Key": OPENAI_API_KEY
  }
)

print(client.is_ready())

## Create Tenant-ready collection

In [ ]:
from weaviate.classes.config import Configure

if (client.collections.exists("Papers")):
    client.collections.delete("Papers")

client.collections.create(
    "Papers",

    vector_config=[
        Configure.Vectors.text2vec_ollama(
            name="main_vector",

            model="nomic-embed-text",
            api_endpoint=OPENAI_URL,
            source_properties=["chunk"]
        )
    ],

    # supported models: https://weaviate.io/developers/weaviate/model-providers/openai/generative#available-models
    generative_config=Configure.Generative.ollama(
        model="llama3.2:1b",
        api_endpoint=OPENAI_URL 
    ),

    multi_tenancy_config=Configure.multi_tenancy(True)
)

## List Tenants

In [ ]:
papers = client.collections.get("Papers")
papers.tenants.get()

## Close the client

In [ ]:
client.close()